# Here’s a road map of what we will do with Pandas:

1. Set up the environment and load the data
2. Investigate the data
3. Parse the different data tabs
4. Standardize existing columns and create new ones
5. Clean up the data using “apply” and “lambda” functions
3. Reshape the data from wide to long by pivoting on multi-level indices and stacking
3. Save the final results back to excel





# 1. Set up the environment and load the data

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Load the raw data using the ExcelFile object
data = pd.ExcelFile('/kaggle/input/reshaping-data/reshaping_data.xlsx')

# 2. Investigate the data

In [ ]:
# First, see the sheetnames available
data.sheet_names

In [ ]:
# Take a peek at the first 10 rows of the first tab
data.parse(sheetname='ABC_inc', skiprows=0).head(10)

In [ ]:
tabnames = data.sheet_names

In [ ]:
i = 0
df = data.parse(sheetname=tabnames[i], skiprows=7)
df.head(2)

# 4. Standardize existing columns and create new ones

In [ ]:
# make a list of the header row and strip up to the 4th letter. This is the location and year information
cols1 = list(df.columns)
cols1 = [str(x)[:4] for x in cols1]

# make another list of the first row,this is the age group information
# we need to preserve this information in the column name when we reshape the data 
cols2 = list(df.iloc[0,:])
cols2 = [str(x) for x in cols2]

# now join the two lists to make a combined column name which preserves our location, year and age-group information
cols = [x+"_"+y for x,y in zip(cols1,cols2)]

# Assign new column names to the dataframe
df.columns = cols
df.head(1)

In [ ]:
# Drop empty columns, Rename the useful columns
# Note when you drop, you should specify axis=1 for columns and axis=0 for rows
df = df.drop(["Unna_nan"], axis=1).iloc[1:,:].rename(columns={'dist_nan':'district', 'prov_nan': 'province', 'part_nan':'partner', 'fund_nan':'financing_source'})
df.head(2)

In [ ]:
# Engineer a new column for the organization, grab this name from the excel tab name
# This should read 'ABC inc' if executed correctly
df['main_organization'] = tabnames[i].split("_")[0] + " "+ tabnames[i].split("_")[1]
df.main_organization.head(2)

**Let’s pause and look at the structure of our dataframe so far.**

In [ ]:
df.info()

# 5. Clean up the data using “apply” and “lambda” functions

In [ ]:
# Make lists of the columns which need attention and use this as reference to execute
# You will notice that I use list comprehension every time I generate an iterable like a list or dictionary
# This is really amazing python functionality and I never want to go back to the old looping way of doing this!
to_remove = [c for c in df.columns if "Total" in c] # redundant
to_change = [c for c in df.columns if "yrs" in c] # numeric

# drop unwanted columns
# Notice that you need to specify inplace, otherwise pandas will return the data frame instead of changing it in place
df.drop(to_remove, axis=1, inplace= True) 

# Change the target column data types
for c in to_change:
    df[c] = df[c].apply(lambda x: pd.to_numeric(x))

In [ ]:
df.info()

# 6. Reshape the data from wide to long by pivoting on multi-level indices and stacking

In [ ]:
# First, select the columns to use for a multi-level index. This depends on your data
# Generally, you want all the identifier columns to be included in the multi-index 
# For this dataset, this is every non-numeric column
idx =['district', 'province', 'partner', 'financing_source', 'main_organization']

# Then pivot the dataset based on this multi-level index 
multi_indexed_df = df.set_index(idx)
multi_indexed_df.head(2)

In [ ]:
# Stack the columns to achieve the baseline long format for the data
stacked_df = multi_indexed_df.stack(dropna=False)
stacked_df.head(25) 

# check out the results!

**Mind. Blown! Exactly how I felt when I saw this for the first time too.**

In [ ]:
# Now do a reset to disband the multi-level index, we only needed it to pivot our data during the reshape
long_df = stacked_df.reset_index()
long_df.head(3)

In [ ]:
# Make series of lists which split year from target age-group
# the .str attribute is how you manipulate the data frame objects and columns with strings in them
col_str = long_df.level_5.str.split("_") 
col_str.head(3)

In [ ]:
# engineer the columns we want, one columns takes the first item in col_str and another columns takes the second 
long_df['target_year'] = [x[0] for x in col_str] 
long_df['target_age'] = [x[1] for x in col_str]
long_df['target_quantity'] = long_df[0] # rename this column
long_df.head(2)

In [ ]:
# drop the now redundant columns
df_final = long_df.drop(['level_5', 0], axis=1)
df_final.head(2)

# 7. Save the final results back to Excel

In [ ]:
df_final.to_excel("reshaping_result_long_format.xlsx")